In [48]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [49]:
train_raw = pd.read_csv('../data/train_MPRA.txt', delimiter='\t', header=None)
test_raw = pd.read_csv('../data/test_MPRA.txt', delimiter='\t', header=None)
train_sol = pd.read_csv('../data/trainsolutions.txt', delimiter='\t', header=None)
train_raw.head()
strand_length = 295


In [50]:
# Get our x and y data
train_scores = np.array(train_raw.iloc[:, 2:297]) #Dimensions are 8000 (samples) by 295 (SHARPR scores per nucleotide)
raw_dna_strands_train = [list(train_raw[1][i]) for i in range(len(train_raw))] #List of lists holding DNA strands separated by character. Size 8000 lists each of length 290
embedded_dna_strands_train = [np.column_stack((np.array(pd.get_dummies(pd.concat([pd.Series(raw_dna_strands_train[i]), pd.Series(["A", "C", "T", "G"])]), dtype='int'))[:-4], np.arange(295))) for i in range(len(train_raw))] #One hot encoded dna strands, list of 8000 matrices, each (295,5)
embedded_dna_strands_train = [embedded_dna_strands_train[i] for i in range(len(embedded_dna_strands_train)) if not ("N" in raw_dna_strands_train[i])]
train_scores  = [train_scores[i] for i in range(len(raw_dna_strands_train)) if not ("N" in raw_dna_strands_train[i])]
#Repeat for test data
raw_dna_strands_test = [list(test_raw[1][i]) for i in range(len(test_raw))] #List of lists holding DNA strands separated by character. Size 8000 lists each of length 290
embedded_dna_strands_test = [np.column_stack((np.array(pd.get_dummies(pd.concat([pd.Series(raw_dna_strands_test[i]), pd.Series(["A", "C", "T", "G"])]), dtype='int'))[:-4], np.arange(295))) for i in range(len(test_raw))]
embedded_dna_strands_test = [embedded_dna_strands_test[i] for i in range(len(embedded_dna_strands_test)) if not ("N" in raw_dna_strands_test[i])]

In [51]:
#Add column with unique identifier for each nucleotide (sequence + location)
train_sol[3] = [str(train_sol.iloc[i, 1][5:]).zfill(4) + str(train_sol.iloc[i,2]).zfill(3) for i in range(len(train_sol))]

#Split by activators and repressors
train_sol_act = train_sol[train_sol[0] == 'A'][3]
train_sol_rep = train_sol[train_sol[0] == 'R'][3]

### ML Model

In [52]:
class DNADataset(Dataset):
    def __init__(self, embedded_dna_strands, train_scores):
        self.x = torch.tensor(embedded_dna_strands, dtype=torch.float32) # Convert x and y to tensors
        self.y = torch.tensor(train_scores, dtype=torch.float32)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [53]:
class SelfAttentionFeedForward(nn.Module):
    #Initialize hyperparameters and NN matrices
    def __init__(self, attention_size, seq_len, embed_size, hidden_size, hidden_layers, lr, train_len):
        super().__init__()
        self.attention_size = attention_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.hidden_layers = hidden_layers
        self.lr = lr
        self.train_len = train_len
        self.seq_len = seq_len
        #self.dropout_rate = dropout_rate 

        self.initAttention()
        self.initFFN()

        self.optimizer = torch.optim.Adam(
            self.parameters(),
            lr=self.lr,
            amsgrad=True,
        )

    #Initialize our weight matrices as torch objects, allows them to be automatically optimized
    def initAttention(self):
        self.W_Q = nn.Linear(self.embed_size, self.attention_size, bias=False)
        self.W_K = nn.Linear(self.embed_size, self.attention_size, bias=False)
        self.W_V = nn.Linear(self.embed_size, self.attention_size, bias=False)

        
    
    #Initialize Feed Forward layers, based on however many hidden layers we want
    def initFFN(self):

        layers = []

        layers.append(nn.Linear(self.attention_size, self.hidden_size))
        layers.append(nn.ReLU())
        #layers.append(nn.Dropout(self.dropout_rate))
        
        for _ in range(self.hidden_layers - 1):
            layers.append(nn.Linear(self.hidden_size, self.hidden_size))
            layers.append(nn.ReLU())
            #layers.append(nn.Dropout(self.dropout_rate)) #Add this later on

        layers.append(nn.Linear(self.hidden_size, 1))

        self.layers = nn.ModuleList(layers)
        self.criterion = nn.MSELoss() # Switch to mean squared error instead of simple norm (this is better apparently?)

    def loss(self, predicted, y):
        return torch.norm(predicted - y)

    def forward(self, x):
        # x of size                                               (batch_size, sequence_length, embedding_size)
        if x.shape[-1] != self.embed_size:
            raise ValueError

        queries = self.W_Q(x) #                                   (batch_size, sequence_length, attention_size)
        keys = self.W_K(x) #                                      (batch_size, sequence_length, attention_size)
        values = self.W_V(x) #                                    (batch_size, sequence_length, attention_size)

        # Scale to prevent overflow errors, divide by square root of attention dimension
        scale = torch.sqrt(torch.tensor(self.attention_size, dtype=torch.float32))

        #Compute attention and then normalize
        attention = torch.bmm(queries, keys.transpose(1,2)) / scale #                  (batch_size, seq_len, seq_len)
        weights = torch.nn.functional.softmax(attention, dim=2) # Apply this per sample

        # Use as weights for values
        context = torch.bmm(weights, values) #    (batch_size, attention_size, sequence_length)
        # Run through all FFN layers
        for layer in self.layers:
            context = layer(context)
        # Return prediction with added b term
        return context.squeeze(-1)

    def train_step(self, x, y):
        self.optimizer.zero_grad()
        pred = self(x)
        loss = self.criterion(pred, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=1.0) # This is for stability
        self.optimizer.step()
        return loss.item() # Diagnostic info

    def train(self, dataloader):
        losses = []
        for epoch in range(self.train_len):
            epoch_loss = 0
            for x_batch, y_batch in dataloader:
                loss = self.train_step(x_batch, y_batch)
                epoch_loss += loss
            avg_loss = epoch_loss / len(dataloader)
            losses.append(avg_loss)
            if (epoch + 1) % 5 == 0:
                print(f"Epoch {epoch+1}/{self.train_len}, Loss: {avg_loss:.4f}")
        return losses
            

In [57]:

dataset = DNADataset(embedded_dna_strands_train, train_scores)

# Create a DataLoader for batching, shuffling, and parallel data loading
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

model = SelfAttentionFeedForward(20, 295, 5, 10, 1, 1e-4, 35) # (attention_size, seq_len, embed_size, hidden_size, hidden_layers, lr, train_len)
model.train(dataloader)


Epoch 5/35, Loss: 0.3566
Epoch 10/35, Loss: 0.3556
Epoch 15/35, Loss: 0.3550
Epoch 20/35, Loss: 0.3545
Epoch 25/35, Loss: 0.3546
Epoch 30/35, Loss: 0.3547
Epoch 35/35, Loss: 0.3552


[0.3722450732588768,
 0.36231695806980135,
 0.35980509281158446,
 0.35866012406349185,
 0.35661341854929923,
 0.3559960229992867,
 0.359088103979826,
 0.35603070840239526,
 0.3574793221652508,
 0.355563128978014,
 0.35619551116228104,
 0.3593996531069279,
 0.35545326939225197,
 0.35616158944368365,
 0.3550125886499882,
 0.35506059423089026,
 0.3565283133685589,
 0.3555122908055782,
 0.3553130387365818,
 0.3545332871377468,
 0.35446419486403463,
 0.3545258333384991,
 0.35608106151223184,
 0.35420405626296997,
 0.35460475540161135,
 0.3538242387771606,
 0.3551491311788559,
 0.35666617238521575,
 0.3551843306422234,
 0.3546984785795212,
 0.3567810878455639,
 0.3567938947081566,
 0.35445902952551844,
 0.35529606401920316,
 0.3552296207249165]

In [63]:
nucleotide_vals = model.forward(torch.Tensor(embedded_dna_strands_test)).detach().numpy()
nucleotide_vals

array([[-0.02421159, -0.00877821, -0.00672451, ..., -0.01295421,
        -0.01295421, -0.01295421],
       [-0.02848384, -0.0219145 , -0.02493575, ..., -0.03607342,
        -0.03607342, -0.03607342],
       [-0.03253882, -0.01874664, -0.0211546 , ..., -0.03607342,
        -0.03607342, -0.03607342],
       ...,
       [-0.02467053, -0.00340417, -0.00290501, ..., -0.01295421,
        -0.01295421, -0.01295421],
       [-0.02452026, -0.00872271, -0.00896282, ...,  0.00754449,
         0.00754488,  0.00754488],
       [-0.03271075, -0.01227978, -0.01104505, ...,  0.00754488,
         0.00754488,  0.00754488]], dtype=float32)

In [71]:
output = []

flat_vals = nucleotide_vals.flatten()
top_100k_indices = np.argpartition(flat_vals, -100000)[-100000:] # Get top 100k indices and sort (sort and get indices using argpartition)
top_100k_indices_sorted = top_100k_indices[np.argsort(flat_vals[top_100k_indices])[::-1]]

bottom_50k_indices = np.argpartition(flat_vals, 50000)[:50000]  # Get bottom 50k indices and sort
bottom_50k_indices_sorted = bottom_50k_indices[np.argsort(flat_vals[bottom_50k_indices])]  

top_100k_coords = [divmod(index, nucleotide_vals.shape[1]) for index in top_100k_indices_sorted]
bottom_50k_coords = [divmod(index, nucleotide_vals.shape[1]) for index in bottom_50k_indices_sorted]

max_row = max(max(row for row, _ in top_100k_coords), 
              max(row for row, _ in bottom_50k_coords))
pad_width = len(str(max_row))

for row, col in top_100k_coords:
    output.append(["A", f"test{str(row).zfill(pad_width)}", f"{col}"])
for row, col in bottom_50k_coords:
    output.append(["R", f"test{str(row).zfill(pad_width)}", f"{col}"])

output_df = pd.DataFrame(output, columns = ["Activation/Repression", "SequenceID", "Nucleotide"])
output_df.to_csv("predictions.txt", sep="\t", header=False, index=False)



In [59]:

#The output should now reflect predictions based on the training data

#Generate predictions
predictions = []
for i, strand in enumerate(embedded_dna_strands_train):
    logits = model.forward(torch.Tensor(strand).unsqueeze(0)).detach().numpy().flatten()
    predicted_scores = torch.sigmoid(torch.Tensor(logits)).numpy().flatten()  #Apply Sigmoid to logits
    predicted_labels = ["A" if score > 0.5 else "R" for score in predicted_scores]  #0.5 threshold
    sequence_id = f"train{str(i).zfill(4)}"
    nucleotides = raw_dna_strands_train[i]

    for pos, (nucleotide, label) in enumerate(zip(nucleotides, predicted_labels)):
        predictions.append([label, sequence_id, nucleotide])

#Save predictions
predictions_df = pd.DataFrame(predictions, columns=["Activation/Repression", "Sequence_ID", "Nucleotide"])
predictions_df.to_csv("predictions.txt", sep="\t", header=False, index=False)

#Load training solutions
training_solutions = pd.read_csv("../data/trainsolutions.txt", delimiter="\t", header=None, names=["Activation/Repression", "Sequence_ID", "Nucleotide"])

#Load predictions
predictions = pd.read_csv("predictions.txt", delimiter="\t", header=None, names=["Activation/Repression", "Sequence_ID", "Nucleotide"])

#Ensure "Nucleotide" columns are strings in both DataFrames
predictions["Nucleotide"] = predictions["Nucleotide"].astype(str)
training_solutions["Nucleotide"] = training_solutions["Nucleotide"].astype(str)

#Merge predictions and training solutions
comparison = pd.merge(predictions, training_solutions, on=["Sequence_ID", "Nucleotide"], how="inner", suffixes=('_pred', '_true'))

#Add a "Match" column to indicate where Prediction matches Truth
comparison["Match"] = comparison["Activation/Repression_pred"] == comparison["Activation/Repression_true"]

#Calculate accuracy
accuracy = comparison["Match"].mean()
print(f"Accuracy: {accuracy:.2%}")

#Save the comparison to a file for inspection
comparison.to_csv("comparison.txt", sep="\t", index=False)

Accuracy: nan%
